# 영화 추천 시스템

- 추천 시스템 : 특정 사용자에 대하여 다양한 정보를 활용하여 원하는 콘텐츠를 제공하는 것

- ex) 유튜브, 넷플릭스, 쇼핑몰, 광고 등

## 콘텐츠 기반 필터링 (TMDB 5000 영화 데이터)

- 사용자가 특정한 아이템을 선호하는 경우, 그 아이템과 비슷한 아이템을 추천하는 방식


### 0. 데이터 준비

In [2]:
# - 데이터 읽기
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies = pd.read_csv('./tmdb_5000_movies.csv')
print(movies.shape)

(4803, 20)


In [3]:
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


### 1. EDA, 피처 선정
- 컬럼 정보
    - id : 영화 아이디
    - title : 영화명
    - genres : 영화 장르
    - vote_average : 영화 평균 평점
    - vote_count : 영화 투표수
    - popularity : 영화 인기
    - keywords : 영화 키워드
    - overview : 영화 개요

※ genres, keywords의 자료형을 확인해보자 !

In [19]:
# 사용할 피처 선정
# 데이터 확인 head(n)
use_columns =['id', 'title', 'genres', 'vote_average', 'vote_count',
              'popularity', 'keywords', 'overview']
movies_df=pd.DataFrame(movies, columns=use_columns)
movies_df.shape

(4803, 8)

In [38]:
# genres의 자료형
movies_df['genres'][3]

[{'id': 28, 'name': 'Action'},
 {'id': 80, 'name': 'Crime'},
 {'id': 18, 'name': 'Drama'},
 {'id': 53, 'name': 'Thriller'}]

In [ ]:
# keywords의 자료형


In [28]:
movies_df['genres'] = movies_df['genres'].apply(eval)

In [14]:
movies_df['keywords'] = movies_df['keywords'].apply(eval)

In [42]:
movies_df['genres'] = movies_df['genres'].apply(lambda genres: [genre_dict['name'] for genre_dict in genres])

genres와 keywords에서 id는 제외

하나의 문자열로 바꾸기 (새로운 feature로 추가)

해당 문자열을 DTM으로 벡터화

In [45]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

KeyError: 'genres_literal'

※ genre_mat의 자료형은 무엇일까?

cosine similarity 계산

In [ ]:
import numpy as np

def cos_similarity(a, b):
    
    return 

cos_similarity(np.array((7.0, 12.4, 256.0, 322.17)), np.array((7.0, 12.4, 256.0, 322.17)))

### 2. 유사도 계산을 통한 추천 시스템 구현
matirx의 코사인 유사도를 계산
- 코사인 유사도 : 두 벡터 간의 cosine 각도를 이용하여 구할 수 있는 두 벡터 간 유사도

In [ ]:
## 직접 계산


In [44]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])

NameError: name 'genre_mat' is not defined

genre_sim에서 특정 영화와 유사도가 높은 순서대로 정렬
- argsort : 값들의 배열에서 데이터를 정렬한 **index** 반환

In [ ]:
# 내림차순 정렬을 위해 -1 옵션을 추가로 준다
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:1])

추천 영화 DataFrame 반환 함수

In [ ]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title']==title_name]

    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]

    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)

    return df.iloc[similar_indexes]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, "The Godfather", 10)
similar_movies[['title', 'vote_average']]

- Vote Average Feature 활용

적은 수의 사람이 투표한 경우, 평점이 정말 유효한지 판단하는 데에는 어려움이 있을 수 있다.

In [ ]:
movies_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10]

영화 선정을 위한 가중치 계산식

- v: 개별 영화에 평점을 투표한 횟수  
- m: 평점을 부여하기 위한 최소 투표 횟수  
- R: 개별 영화에 대한 평균 평점  
- C: 전체 영화에 대한 평균 평점  

최소 투표 횟수를 전체의 60% 지점으로 지정

In [ ]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']

    return ((v/(v+m)) * R) + ((m / (v + m)) * C)

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)
movies_df.head(2)

가중치평점을 적용한 추천 영화 DataFrame 반환 함수

In [43]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values

    similar_indexes = sorted_ind[title_index, : (top_n * 2)]
    similar_indexes = similar_indexes.reshape(-1)

    similar_indexes = similar_indexes[similar_indexes != title_index]

    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

NameError: name 'genre_sim_sorted_ind' is not defined

## 아이템 기반 최근접 이웃 협업 필터링 (MovieLens 리뷰 데이터)

### 0. 데이터 준비

In [46]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies = pd.read_csv('./movielens/movies.csv')
ratings = pd.read_csv('./movielens/ratings.csv')
movies.shape, ratings.shape

((9742, 3), (100836, 4))

In [47]:
movies.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [48]:
ratings.head(2)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247


### 1. EDA, 데이터 정리

- ratings와 movie를 movieId를 기준으로 merge

In [51]:
rating_movies = pd.merge(movies, ratings, on='movieId')

In [52]:
rating_movies[:2]

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962


pivot table를 활용하여, rating에 대하여 userId, title로 이루어진 데이터프레임으로 변환

In [56]:
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')
ratings_matrix.fillna(0, inplace=True)
ratings_matrix.head(2)


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


영화에 대한 리뷰 갯수를 벡터로 사용하기 위해 transpose

In [59]:
ratings_matrix_T = ratings_matrix.T

In [60]:
ratings_matrix_T.head(2) 

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


코사인 유사도 측정

In [62]:
from sklearn.metrics.pairwise import cosine_similarity

item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns, columns=ratings_matrix.columns)

print(item_sim_df.shape)
item_sim_df

(9719, 9719)


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.141653,0.000000,...,0.000000,0.342055,0.543305,0.707107,0.0,0.000000,0.139431,0.327327,0.000000,0.0
'Hellboy': The Seeds of Creation (2004),0.000000,1.000000,0.707107,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
'Round Midnight (1986),0.000000,0.707107,1.000000,0.000000,0.000000,0.0,0.176777,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
'Salem's Lot (2004),0.000000,0.000000,0.000000,1.000000,0.857493,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
'Til There Was You (1997),0.000000,0.000000,0.000000,0.857493,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ (1999),0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.211467,0.216295,0.097935,0.132489,...,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.192259,0.000000,0.170341,0.0
xXx (2002),0.139431,0.000000,0.000000,0.000000,0.000000,0.0,0.089634,0.000000,0.276512,0.019862,...,0.069716,0.305535,0.173151,0.246482,0.0,0.192259,1.000000,0.270034,0.100396,0.0
xXx: State of the Union (2005),0.327327,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.156764,0.000000,...,0.000000,0.382543,0.177838,0.231455,0.0,0.000000,0.270034,1.000000,0.000000,0.0


추천영화 DataFrame을 반환하는 함수

In [ ]:
def find_sim_movie_item(df, title_name, top_n=10):
    title_movie_sim = df[[title_name]].drop(title_name, axis=0)

    return title_movie_sim.sort_values(title_name, ascending=False)[:top_n]

In [ ]:
find_sim_movie_item(item_sim_df, 'Godfather, The (1972)')

In [ ]:
find_sim_movie_item(item_sim_df, 'Inception (2010)')